In [ ]:
import json
import os
from library_dicom.dicom_processor.model.Series import Series 
from library_dicom.dicom_processor.model.SeriesPT import SeriesPT
from library_dicom.dicom_processor.model.SeriesCT import SeriesCT
from library_dicom.dicom_processor.model.csv_reader.MaskBuilder import MaskBuilder

from library_dicom.dicom_processor.tools.folders import *
from library_dicom.dicom_processor.tools.create_mip import *
from library_dicom.dicom_processor.tools.threshold_mask import *

import numpy as np
import SimpleITK as sitk
import csv

#### - Get dataset from  PARSE_FILTERED_DICOM_DATASET script

In [ ]:
#LIST_PATH from json
json_path = '/media/deeplearning/LACIE SHARE/relevance_dataset.json'
dataset = []
with open(json_path) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        dataset.append(info)


In [ ]:
number_of_study = len(dataset)
print("number of study : ", number_of_study)

#### - Add CSV file for every study on the dataset

In [ ]:
#add csv_file from csv 

csv_path= '/home/deeplearning/AHL/AHL_CSV_DATA/pet0.csv'
csv_file = []
with open(csv_path, 'r') as csv_file :
    reader = csv.reader(csv_file, delimiter = ';') #liste pour chaque ligne 
    csv_file = []
    for row in reader :
        csv_file.append(row)
        
del csv_file[0] #enlever première ligne

In [ ]:
dataset[0]

In [ ]:
#add csv_file from a list 

csv_path = '/media/deeplearning/LACIE SHARE/RELEVANCE Fichier CSV'
liste_csv_file = os.listdir(csv_path)
liste_csv = []
for csv_file in liste_csv_file : 
    if "csv" in csv_file : 
        liste_csv.append(os.path.join(csv_path, csv_file))

#print(liste_csv)
for data in dataset : 
    #nom = '_'.join(data[-1].split('_')[0:2])
    #print(nom)
    patient_id = data[-1]
    for csv_f in liste_csv : 
        if patient_id in csv_f: 
            data.append(csv_f)

#### - Verify if every study has a csv file, 
####            If not, write a json , remove study from dataset

In [ ]:
no_csv = []
for data in dataset : 
    if len(data) == 8 : 
        no_csv.append(data)
print("Nbr de study sans csv :", len(no_csv))
write_json_file('/media/deeplearning/LACIE SHARE/RELEVANCE Fichier CSV', 'study_no_csv', no_csv)
try : 
    for err in no_csv : 
        dataset.remove(err)
except Exception as err : 
    print(err) 

#### - If a study has more than 1 CSV, keep CSV with higher volume

In [ ]:
dataset[0]

In [ ]:
from library_dicom.dicom_processor.tools.csv_volume import *
for data in dataset : 
    liste_csv = []
    if len(data) != 9 : #more than 1 csv 
        for item in data : 
            if 'csv' in item : 
                liste_csv.append(item)
        #print(liste_csv)
        wrong_csv = keep_csv_higher_volume(liste_csv)
        #print("remove : ", remove)
        try : 
            for r in wrong_csv : 
                data.remove(r)
        except Exception as err : 
            print(err)

In [ ]:
#check 
for data in dataset : 
    if len(data) != 9 : 
        print(data)

#### -  Rewrite path and save complete dataset as json file 

In [ ]:
#rewrite path 

for data in dataset : 
    new_1 = data[0].replace('/media/oncopole/DD 2To', '/media/deeplearning/Elements')
    new_2 = data[2].replace('/media/oncopole/DD 2To', '/media/deeplearning/Elements')
    data[0] = new_1
    data[2] = new_2

In [ ]:
print('Nombre de study : ', len(dataset))

In [ ]:
write_json_file('/media/deeplearning/LACIE SHARE', 'RELEVANCE_validated_dataset', dataset)

#### - Remove remove_bi_file if it exists in every serie 

In [ ]:
from library_dicom.dicom_processor.tools.folders import *
directory_validated_dicom = ''
paths = get_series_path(directory_validated_dicom)

for path in paths : 
    remove_bi_file(path)

### Different scripts : 

#### - Check if there is Unconstance Spacing error in every study of the dataset 

In [ ]:
unconstant_spacing = []
serie_error = []
for serie in dataset : 
    print(dataset.index(serie))
    try : 
        if serie[1] =='PT' : 
            serie_pt_objet = SeriesPT(serie[0])
            nifti_array = serie_pt_objet.get_numpy_array()
            if serie_pt_objet.get_z_spacing =='Unconstant Spacing' :
                print("Unconstant Spacing PET")
                unconstant_spacing.append(serie)
            
            serie_ct_objet = SeriesCT(serie[2])
            nifti_array_ct = serie_ct_objet.get_numpy_array()
            if serie_ct_objet.get_z_spacing =='Unconstant Spacing' :
                print("Unconstant Spacing CT")
                unconstant_spacing.append(serie)

        else : 
            serie_pt_objet = SeriesPT(serie[2])
            nifti_array = serie_pt_objet.get_numpy_array()
            if serie_pt_objet.get_z_spacing =='Unconstant Spacing' :
                print("Unconstant Spacing PT")
                unconstant_spacing.append(serie)
            serie_ct_objet = SeriesCT(serie[0])
            nifti_array_ct = serie_ct_objet.get_numpy_array()
            if serie_ct_objet.get_z_spacing =='Unconstant Spacing' :
                print("Unconstant Spacing CT")
                unconstant_spacing.append(serie)

    except Exception as err : 
        print(serie)
        print(err)
        serie_error.append(serie)


#### - Generate PET, CT, MASK NIFTI with checking suv_max, mean and sd

In [ ]:
#Generate PET, CT, MASK NIFTI with checking suv_max, mean and sd

nifti_directory = '/media/deeplearning/LACIE SHARE/RELEVANCE/NIFTI'
mip_directory = '/media/deeplearning/LACIE SHARE/RELEVANCE/MIP'

#save serie_path with false mask 
serie_false_mask = []
#save result about serie with false mask 
results_false_mask = []
#save path of MIP to generate PDF 
path_mip = []

#save error serie 
error_dataset = []

for serie in dataset: 
    try : 
        print(dataset.index(serie))
        subliste = []
        if serie[1] == 'PT' : 
            
            serie_pt_objet = SeriesPT(serie[0])
            
            serie_pt_objet.get_instances_ordered()
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
            size_matrix = serie_pt_objet.get_size_matrix()
            mask_objet = MaskBuilder(serie[-1], size_matrix)
            mask_4D = mask_objet.mask_array

            number_roi = mask_4D.shape[3]
            threshold = mask_objet.details_rois['SUVlo']

            if mask_objet.is_correct_suv(nifti_array) == True : 

                print("MASK CORRECT")
                #generation nifti PT
                filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                #serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                print("EXPORT NIFTI PT")

                #generation nifti CT
                serie_ct_objet = SeriesCT(serie[2])
                serie_ct_objet.get_instances_ordered()
                serie_ct_objet.get_numpy_array()
                filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                #serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                print("EXPORT NIFTI CT")

                #generation nifti mask
                filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_4D)
                print("EXPORT NIFTI MASK")


            else : 
                results = []
                print("FALSE MASK")
                serie_false_mask.append(serie)

                print(mask_objet.calcul_suv(nifti_array))
                results.append(mask_objet.calcul_suv(nifti_array))

                print(mask_objet.ecart_suv_max(nifti_array))
                results.append(mask_objet.ecart_suv_max(nifti_array))

                print(mask_objet.ecart_suv_mean(nifti_array))
                results.append(mask_objet.ecart_suv_mean(nifti_array))

                print(mask_objet.ecart_SD(nifti_array))
                results.append(mask_objet.ecart_SD(nifti_array))

                results_false_mask.append(results)

                #threshold mask 41% 
                mask_4D = threshold_matrix(mask_4D, nifti_array, 0.41)

                #create mip for false mask and check 
                #pet
                angle_filename = mip_projection(nifti_array, 0, mip_directory, study_uid, 'pet', cmap='Greys', vmin=0, vmax=7) 
                subliste.append(angle_filename)
                print('MIP PET')
               # mask
                angle_filename_mask = mip_projection_4D(mask_4D, 0, mip_directory, study_uid, number_roi, cmap='Greys')
                subliste.append(angle_filename_mask)
                print('MIP MASK')
                path_mip.append(subliste)


        else : 
            serie_pt_objet = SeriesPT(serie[2])
            serie_pt_objet.get_instances_ordered()
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
            size_matrix = serie_pt_objet.get_size_matrix()
            mask_objet = MaskBuilder(serie[-1], size_matrix)
            mask_4D = mask_objet.mask_array

            number_roi = mask_4D.shape[3]
            threshold = mask_objet.details_rois['SUVlo']

            if mask_objet.is_correct_suv(nifti_array) == True :  
                print("MASK CORRECT")
                #generation nifti PT
                filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                #serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                print("EXPORT NIFTI PT")

                #generation nifti CT
                serie_ct_objet = SeriesCT(serie[0])
                serie_ct_objet.get_instances_ordered()
                serie_ct_objet.get_numpy_array()
                filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                #serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                print("EXPORT NIFTI CT")

                #generation nifti mask
                filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_4D)
                print("EXPORT NIFTI MASK")



            else : 
                results = []
                print("FALSE MASK")
                serie_false_mask.append(serie)

                print(mask_objet.calcul_suv(nifti_array))
                results.append(mask_objet.calcul_suv(nifti_array))

                print(mask_objet.ecart_suv_max(nifti_array))
                results.append(mask_objet.ecart_suv_max(nifti_array))

                print(mask_objet.ecart_suv_mean(nifti_array))
                results.append(mask_objet.ecart_suv_mean(nifti_array))

                print(mask_objet.ecart_SD(nifti_array))
                results.append(mask_objet.ecart_SD(nifti_array))

                results_false_mask.append(results)

                #threshold mask 41% 
                mask_4D = threshold_matrix(mask_4D, nifti_array, 0.41)

                #create mip for false mask and check 
                #pet
                angle_filename = mip_projection(nifti_array, 0, mip_directory, study_uid, 'pet', cmap='Greys', vmin=0, vmax=7) 
                subliste.append(angle_filename)
                print("MIP PET")
                #mask
                angle_filename_mask = mip_projection_4D(mask_4D, 0, mip_directory, study_uid, number_roi, cmap='Greys')
                subliste.append(angle_filename_mask)
                print('MIP MASK ')
                path_mip.append(subliste)
               

    except Exception as err : 
        print(serie)
        print(err)
        error_dataset.append(serie)

In [ ]:
print("number of error : ", len(error_dataset))
print("number of false mask : " , len(serie_false_mask))

In [ ]:
#Save different json 
directory_nifti = '/media/deeplearning/LACIE SHARE/RELEVANCE'
write_json_file(directory_nifti, 'false_mask_serie', serie_false_mask)
write_json_file(directory_nifti, 'false_mask_results', results_false_mask)
#write_json_file(directory_nifti, 'error', error_dataset)


#### - Generate PET, CT, MASK NIFTI without checking

In [ ]:
json_path = ''
dataset = []
with open(json_path) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        dataset.append(info)

In [ ]:
nifti_directory = '/media/deeplearning/LACIE SHARE/RELEVANCE/NIFTI'

error_dataset = []

for serie in dataset: 
    try : 
        print(dataset.index(serie))

        if serie[1] == 'PT' : 
            serie_pt_objet = SeriesPT(serie[0])
            serie_pt_objet.get_instances_ordered()
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
            size_matrix = serie_pt_objet.get_size_matrix()
            mask_objet = MaskBuilder(serie[-1], size_matrix)
            mask_4D = mask_objet.mask_array

            number_roi = mask_4D.shape[3]

            print("MASK CORRECT")
                    #generation nifti PT
            filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
            #serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
            print("EXPORT NIFTI PT")

                    #generation nifti CT
            serie_ct_objet = SeriesCT(serie[2])
            serie_ct_objet.get_instances_ordered()
            serie_ct_objet.get_numpy_array()
            filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
            #serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
            print("EXPORT NIFTI CT")

                    #generation nifti mask
            filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
            serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_4D)
            print("EXPORT NIFTI MASK")
          
          
        else : 
            serie_pt_objet = SeriesPT(serie[2])
            serie_pt_objet.get_instances_ordered()
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
            size_matrix = serie_pt_objet.get_size_matrix()
            mask_objet = MaskBuilder(serie[-1], size_matrix)
            mask_4D = mask_objet.mask_array
            number_roi = mask_4D.shape[3]

            print("MASK CORRECT")
                    #generation nifti PT
            filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
            #serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
            print("EXPORT NIFTI PT")

                     #generation nifti CT
            serie_ct_objet = SeriesCT(serie[0])
            serie_ct_objet.get_instances_ordered()
            serie_ct_objet.get_numpy_array()
            filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
            #serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
            print("EXPORT NIFTI CT")

                    #generation nifti mask
            filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
            serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_4D)
            print("EXPORT NIFTI MASK")
           
    except Exception as err : 
        print(err)
        print(serie)
        error_dataset.append(serie)

In [ ]:
print("number of error : ", len(error_dataset))

#### - Generate PET & CT NIFTI only

In [ ]:
nifti_directory = '/media/deeplearning/LACIE SHARE/Olivier_Morel_sarcome'

error = []
for serie in dataset : 
    print(dataset.index(serie))
    try : 
        if serie[1] == 'PT' : 
                serie_pt_objet = SeriesPT(serie[0]) 
                serie_pt_objet.get_instances_ordered()
                nifti_array = serie_pt_objet.get_numpy_array()
                study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']

                                #generation nifti PT
                filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                print("EXPORT NIFTI PT")
                                #generation nifti CT
                serie_ct_objet = SeriesCT(serie[2])
                serie_ct_objet.get_instances_ordered()
                serie_ct_objet.get_numpy_array()
                filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                print("EXPORT NIFTI CT")

        else : 
                serie_pt_objet = SeriesPT(serie[2])
                serie_pt_objet.get_instances_ordered() 
                nifti_array = serie_pt_objet.get_numpy_array()
                study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']

                        
                                #generation nifti PT
                filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                print("EXPORT NIFTI PT")
                                #generation nifti CT
                serie_ct_objet = SeriesCT(serie[0])
                serie_ct_objet.get_instances_ordered()
                serie_ct_objet.get_numpy_array()
                filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                print("EXPORT NIFTI CT")
 

    except Exception as err : 
        print(serie)
        print(err)
        error.append(serie)